#### Import libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
import tensorflow as tf
import requests
from datetime import datetime
from datetime import timedelta, date
import time
import json

## Task 1

#### Creating a list of dates that takes non public holiday and weekend days (according to https://www.geovictoria.com/mx/recursos-humanos/dias-no-laborales/ and https://www.gob.mx/cenace/articulos/cenace-publica-sus-dias-inhabiles-para-2022?idiom=es)

In [249]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

def create_urls(start_date,end_date):
    # taking care of public holidays
    public_holidays = ['2021-01-01','2021-02-01','2021-03-15','2021-04-01','2021-04-02','2021-05-05','2021-09-16-','2021-10-12','2021-11-02',
                   '2021-11-15','2022-01-01','2022-02-07','2022-03-21','2022-04-14','2022-04-15']

    labor_days = []

    weekdays = [5,6]
    for dt in daterange(start_date, end_date):
        if (dt.weekday() not in weekdays) and (str(dt) not in public_holidays):
            labor_days.append(dt.strftime("%Y-%m-%d"))

    urls_list = []

    for date in labor_days:

        day = date.split('-')[2].replace('0','')
        month = date.split('-')[1]
        year = date.split('-')[0]
        urls_list.append(f'http://www.economia-sniim.gob.mx/Consolidados.asp?prod=&punto=100&edo=&dqdia={day}&dqmes={month}&dqanio={year}&aqdia={day}&aqmes={month}&aqanio={year}')
    
    return urls_list

#### Extracting contents

In [265]:
# function to return the table with the most table data cells
def max_len_table(x):
    max=0
    for i in range(len(x)):
        current_len = len(x[i])
        if current_len > max:
            max = current_len
            max_index = i
    
    return x[max_index].find_all('td')

In [266]:
# function to return the start and stop index
def start_stop(x):
    for i in range(len(x)):
        if START_KEYWORD in ''.join(x[i].findAll(text = True)):
            ini = i+1
        else:
            pass
        if STOP_KEYWORD_1 in ''.join(x[i].findAll(text = True)) or STOP_KEYWORD_2 in ''.join(x[i].findAll(text = True)):
            fin = i
            break
    return ini,fin

In [267]:
# function to clean product name tags text
def clean_string(text):
    return text.replace('\n','').replace('\xa0','')

In [6]:
# Keywords to return start and finish indexes
START_KEYWORD = 'DistribOrig'
STOP_KEYWORD_1 = 'Granos y Semillas'
STOP_KEYWORD_2 = 'Flores'

# Dictionary keys
KEYS_LIST = ['product','min_price','max_price','avg_price','origin','distr']

In [7]:
def main():

    start_dt = date(2021,1,1)
    end_dt = date(2022,7,31)
    urls_list = create_urls(start_dt,end_dt)
    dict_list = []

    for item in urls_list:
        page = requests.get(item)
        if page.status_code != 200:
            page.raise_for_status()
        soup = BeautifulSoup(page.text,'html.parser')
        try:
            soup.find('p').getText()
        except:
            print('Current page contains information.')
        else:
            print('Current page does not contain information.')
            continue
        tables = soup.find_all('table')
        mt = max_len_table(tables)
        start,stop = start_stop(mt)

        dummy_list = []
        for i in range(start,stop):
            dummy_list.append(clean_string(''.join(mt[i].getText())))

        dummy_dict={}
        for i in range(0,len(dummy_list),6):
            dummy_dict = {}
            for j in range(6):
                dummy_dict[KEYS_LIST[j]] = dummy_list[i+j]
            dict_list.append(dummy_dict)
    
    with open('task_1.json','w') as fp:
        json.dump(dict_list,fp,indent=2)
        
    return dict_list

## Task 2.1

In [2]:
f = open('task_1.json')
dict_list = json.load(f)

In [3]:
df = pd.DataFrame(dict_list)
df = df.astype({
                'product':'string',
                'min_price':'float',
                'max_price':'float',
                'avg_price':'float',
                'origin':'string',
                'distr':'string'
                })

In [4]:
df.groupby('product').agg({'describe'})['avg_price']

describe                                                    \
                     count       mean        std    min    25%    50%    75%   
product                                                                        
Acelga               354.0  10.759887   2.988453   6.00   9.00  10.00  12.00   
Aguacate Hass        355.0  58.818310  23.103333  21.11  44.44  47.78  72.22   
Ajo Morado           355.0  52.481690  11.644769  36.00  45.00  50.00  56.00   
Apio                 355.0   6.235690   0.758319   5.00   5.63   6.25   6.88   
Brócoli              355.0  12.238028   3.943863   8.00  10.00  12.00  13.00   
...                    ...        ...        ...    ...    ...    ...    ...   
Uva Superior          52.0  58.197115   9.709044  43.75  50.00  56.25  68.75   
Uva sin semilla      287.0  68.883101   9.752816  43.75  63.75  70.00  75.00   
Zanahoria leña       355.0   2.857465   0.460250   1.60   2.60   2.80   3.20   
Zanahoria mediana    355.0   5.494141   0.494913   4.41   5.13   5.33   5.74   
Zanahoria polvo      355.0   2.780282   0.498988   1.80   2.40   2.80   3.20   

                           
                      max  
product                    
Acelga              20.00  
Aguacate Hass      111.11  
Ajo Morado          94.00  
Apio                 8.13  
Brócoli             29.00  
...                   ...  
Uva Superior        71.25  
Uva sin semilla     85.00  
Zanahoria leña       4.40  
Zanahoria mediana    7.17  
Zanahoria polvo      4.40  

[85 rows x 8 columns]

## Task 2.2

In [6]:
df_avocado = df[df['product']=='Aguacate Hass'].reset_index(drop=True).reset_index()
df_avocado = df_avocado.rename(columns={"index":"time"})
df_avocado = df_avocado.drop(columns=['product','origin','distr'])

In [7]:
df_avocado

,time,min_price,max_price,avg_price
0,0,18.89,23.33,21.11
1,1,20.00,23.33,21.11
2,2,20.00,23.33,21.11
3,3,20.00,23.33,21.11
4,4,21.11,24.44,22.22
...,...,...,...,...
350,350,68.89,75.56,72.22
351,351,61.11,72.22,66.67
352,352,61.11,72.22,65.56
353,353,53.33,63.33,55.56


In [7]:
# func to plot metrics on train and test set
def plot_metrics(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [8]:
# func to plot time series
def plot_series(time, series, ylabel ,format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel(ylabel)
    plt.grid(True)

In [9]:
# func to normalize series
def normalize_series(data,min,max):
    data = data - min
    data = data / max
    return data

In [10]:
# func to create windowed dataset
def windowed_dataset(series, batch_size, n_past, n_future, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [104]:
def task_2_2(N):
    # Creating the main dataframe that we will be transforming to use in the model
    df_avocado = df[df['product']=='Aguacate Hass'].reset_index(drop=True).reset_index()
    df_avocado = df_avocado.rename(columns={"index":"time"})
    df_avocado = df_avocado.drop(columns=['product','origin','distr'])

    # Number of features in the dataset
    N_FEATURES = len(df_avocado.columns)

    # Normalize the data
    data = df_avocado.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Split the data into train and test sets. As test set is pretty important here,
    # we will do a 50/50 split and try to create a model that generalizes well on the test set
    SPLIT_TIME = int(len(data)*0.5)
    x_train = data[:SPLIT_TIME]
    x_test = data[SPLIT_TIME:]

    # Small batch size hence the small amount of data
    BATCH_SIZE = 16

    # Number of past time steps based on which future observations should be predicted.
    N_PAST = N

    # Number of future time steps which are to be predicted.
    N_FUTURE = N

    # Positions from which the window slides to create a new window
    SHIFT = 1

    # Creating a callback to tune the learning rate
    #lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
    # It look like the optimal learning rate was around 0.001

    # Creation of windowed train and test set
    train_set = windowed_dataset(series = x_train, batch_size = BATCH_SIZE,
                                 n_past = N_PAST, n_future = N_FUTURE,
                                 shift = SHIFT)

    test_set = windowed_dataset(series = x_test, batch_size = BATCH_SIZE,
                                n_past = N_PAST, n_future = N_FUTURE,
                                shift = SHIFT)
    
    model = tf.keras.models.Sequential([

        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True,
                                      input_shape = [N_PAST,N_FEATURES], batch_size = BATCH_SIZE)),
        
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Dense(30, activation='relu'),

        tf.keras.layers.Dense(10, activation='relu'),

        tf.keras.layers.Dense(N_FEATURES)

    ])

    EPOCHS = 30

    model.compile(loss = tf.keras.losses.Huber(),
                  optimizer = tf.keras.optimizers.Adam(lr = 0.001),
                  metrics = ["mse","accuracy"])
    
    history = model.fit(train_set, validation_data = test_set, epochs = EPOCHS) #callbacks = [lr_schedule]) was used to tune the learning rate

    # Plotting accuracy
    plt.figure(figsize=(10, 6))
    plot_metrics(history, "accuracy")

    # Plotting mse
    plt.figure(figsize=(10, 6))
    plot_metrics(history, "mse")

    return model,x_test

#### With this, we now know that our learning rate should stay somewhere around 10^-2. But it is important to note that, for this case, it was trained for only four steps into the future; the further we want to predict into the future, the more likely it is for the mse to increase. Either way, the model can be tested with different N future steps.

In [90]:
df_avocado = df[df['product']=='Aguacate Hass'].reset_index(drop=True).reset_index()
df_avocado = df_avocado.rename(columns={"index":"time"})
df_avocado = df_avocado.drop(columns=['product','origin','distr'])
df_avocado

,time,min_price,max_price,avg_price
0,0,18.89,23.33,21.11
1,1,20.00,23.33,21.11
2,2,20.00,23.33,21.11
3,3,20.00,23.33,21.11
4,4,21.11,24.44,22.22
...,...,...,...,...
350,350,68.89,75.56,72.22
351,351,61.11,72.22,66.67
352,352,61.11,72.22,65.56
353,353,53.33,63.33,55.56


In [91]:
355/2

177.5

In [26]:
# func to normalize series
def normalize_series(data,min,max):
    data = data - min
    data = data / max
    return data

def unnormalize_series(data,min,max):
    data = data * max
    data = data + min
    return data

In [247]:
n = 4

# Creating the main dataframe that we will be transforming to use in the model
df_avocado = df[df['product']=='Aguacate Hass'].reset_index(drop=True).reset_index()
df_avocado = df_avocado.rename(columns={"index":"time"})
df_avocado = df_avocado.drop(columns=['product','origin','distr'])

    # Number of features in the dataset
N_FEATURES = len(df_avocado.columns)

# Normalize the data
data = df_avocado.values
# Save values to de normalize data
data_min,data_max = data.min(axis=0),data.max(axis=0)
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Split the data into train and test sets. As test set is pretty important here,
    # we will do a 50/50 split and try to create a model that generalizes well on the test set
SPLIT_TIME = int(len(data)*0.8)
x_train = data[:SPLIT_TIME]
x_test = data[SPLIT_TIME:]

    # Small batch size hence the small amount of data
BATCH_SIZE = 16

    # Number of past time steps based on which future observations should be predicted.
N_PAST = n

    # Number of future time steps which are to be predicted.
N_FUTURE = n

    # Positions from which the window slides to create a new window
SHIFT = 1

    # Creating a callback to tune the learning rate
    #lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
    # It look like the optimal learning rate was around 0.001

    # Creation of windowed train and test set
train_set = windowed_dataset(series = x_train, batch_size = BATCH_SIZE,
                                 n_past = N_PAST, n_future = N_FUTURE,
                                 shift = SHIFT)

test_set = windowed_dataset(series = x_test, batch_size = BATCH_SIZE,
                                n_past = N_PAST, n_future = N_FUTURE,
                                shift = SHIFT)
    
model = tf.keras.models.Sequential([

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True,
                                      input_shape = [N_PAST,N_FEATURES], batch_size = BATCH_SIZE)),
        
    tf.keras.layers.Dropout(0.2),
        
    tf.keras.layers.Dense(30, activation='relu'),

    tf.keras.layers.Dense(10, activation='relu'),

    tf.keras.layers.Dense(N_FEATURES)

    ])

EPOCHS = 30

model.compile(loss = tf.keras.losses.Huber(),
              optimizer = tf.keras.optimizers.Adam(lr = 0.001),
              metrics = ["mse","accuracy"],
              run_eagerly=True)
    
history = model.fit(train_set, validation_data = test_set, epochs = EPOCHS) #callbacks = [lr_schedule]) was used to tune the learning rate

Epoch 1/30


c:\Users\Sebastian\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


18/18 [==============================] - 2s 70ms/step - loss: 0.0321 - mse: 0.0641 - accuracy: 0.6986 - val_loss: 0.1366 - val_mse: 0.2732 - val_accuracy: 1.0000
Epoch 2/30
18/18 [==============================] - 1s 42ms/step - loss: 0.0118 - mse: 0.0237 - accuracy: 0.6977 - val_loss: 0.0540 - val_mse: 0.1081 - val_accuracy: 1.0000
Epoch 3/30
18/18 [==============================] - 1s 42ms/step - loss: 0.0072 - mse: 0.0145 - accuracy: 0.6977 - val_loss: 0.0317 - val_mse: 0.0633 - val_accuracy: 1.0000
Epoch 4/30
18/18 [==============================] - 1s 42ms/step - loss: 0.0051 - mse: 0.0102 - accuracy: 0.6977 - val_loss: 0.0223 - val_mse: 0.0447 - val_accuracy: 1.0000
Epoch 5/30
18/18 [==============================] - 1s 42ms/step - loss: 0.0038 - mse: 0.0077 - accuracy: 0.6977 - val_loss: 0.0167 - val_mse: 0.0334 - val_accuracy: 1.0000
Epoch 6/30
18/18 [==============================] - 1s 42ms/step - loss: 0.0033 - mse: 0.0066 - accuracy: 0.6715 - val_loss: 0.0144 - val_mse: 0.0

In [225]:
# serie de training.... La vamos a utilizar para predecir los proximos N dias....
df_avocado.iloc[282:284]

,time,min_price,max_price,avg_price
282,282,78.89,85.56,82.22
283,283,80.00,86.67,83.33


In [258]:
df_avocado.iloc[282:293].tail()

,time,min_price,max_price,avg_price
288,288,81.11,87.78,83.33
289,289,82.22,88.89,83.33
290,290,83.33,88.89,84.44
291,291,86.67,92.22,87.78
292,292,86.67,92.22,87.78


#### Predicting N time steps into the future

In [ ]:
starting_data = x_train[SPLIT_TIME-2:] 
w_start_set = windowed_dataset(series = starting_data, batch_size = BATCH_SIZE,
                               n_past = N_PAST, n_future = N_FUTURE,
                               shift = 1)

for i in range(N):
        if i != N-1:
            temp_pred = history.model.predict(w_start_set)
            temp_pred = temp_pred.reshape(temp_pred.shape[0],-1)
            temp_arr = np.vstack((starting_data[-1],temp_pred))
            w_start_set = windowed_dataset(series = temp_arr[-2:], batch_size = BATCH_SIZE,
                                           n_past = N_PAST, n_future = N_FUTURE,
                                           shift = 1)
        else:
            future_pred = history.model.predict(w_start_set)
            future_pred = future_pred.reshape(future_pred.shape[0],-1)
            future_pred = unnormalize_series(future_pred,data_min,data_max)
            print('Estimated maximum and minimum price of avocado in',N,'days:', round(future_pred[0][2],2),'$,',round(future_pred[0][1],2), '$')

#### With this, we now know that our learning rate should stay somewhere around 10^-2. But it is important to note that, for this case, it was trained for only four steps into the future; the further we want to predict into the future, the more likely it is for the mse to increase. Either way, the model can be tested with different N future steps.